# `i3wm`

In [1]:
from subprocess import check_output
import i3ipc
i3 = i3ipc.Connection()

# Print the name of the focused window
focused = i3.get_tree().find_focused()
print('Focused window %s is on workspace %s' %
      (focused.name, focused.workspace().name))

Focused window 03-i3wm - Jupyter Notebook - Google Chrome is on workspace 2


In [2]:
def window_is_visible(w):
    try:
        xprop = check_output(['xprop', '-id', str(w.window)]).decode()
    except FileNotFoundError:
        raise SystemExit("The `xprop` utility is not found!" " Please install it and retry.")
    return '_NET_WM_STATE_HIDDEN' not in xprop

def get_ws_windows(conn):
    """ Returns a list of windows from the currently focused workspace. """
    tree = conn.get_tree()
    ws_windows = tree.find_focused().workspace().leaves()
    ws_visible_windows = list(filter(window_is_visible, ws_windows))
    return ws_visible_windows  

In [3]:
test_win = get_ws_windows(i3)
print([w.id for w in test_win])

[94648427380528, 94648427255968]


In [20]:
# Dimensions
def get_ws_dimensions(x, y, conn):
    width, height = None, None
    
    for output in i3.get_outputs():
        if output['active']:
            width = output['rect']['width']
            height = output['rect']['height']
            break
    
    assert width and height
    assert (0 <= x <= width) and (0 <= y <= height)
    return (width, height)

get_ws_dimensions(0, 0, i3)

(1280, 800)

In [5]:
# Bounding boxes
def get_win_boxes(windows):
    win_info = []
    for w in windows:
        w_info = {
            'id': w.id,
            'x': w.rect.x,
            'y': w.rect.y,
            'width': w.rect.width,
            'height': w.rect.height,
            'cx': w.rect.x + (w.rect.width // 2),
            'cy': w.rect.y + (w.rect.height // 2),
            'name': w.name,
            'floating': w.floating.endswith('on')
        }
        win_info.append(w_info)
        
    # sort floating first
    
    # 
    
    return win_info

In [6]:
get_win_boxes(test_win)

[{'id': 94648427380528,
  'x': 0,
  'y': 29,
  'width': 1280,
  'height': 771,
  'cx': 640,
  'cy': 414,
  'name': '03-i3wm - Jupyter Notebook - Google Chrome',
  'floating': False},
 {'id': 94648427255968,
  'x': 109,
  'y': 133,
  'width': 1123,
  'height': 538,
  'cx': 670,
  'cy': 402,
  'name': 'tmux',
  'floating': True}]

In [7]:
get_win_boxes(i3.get_tree().leaves())

[{'id': 94648427461952,
  'x': 2,
  'y': 31,
  'width': 1276,
  'height': 767,
  'cx': 640,
  'cy': 414,
  'name': 'pysight - Visual Studio Code',
  'floating': False},
 {'id': 94648427380528,
  'x': 0,
  'y': 29,
  'width': 1280,
  'height': 771,
  'cx': 640,
  'cy': 414,
  'name': '03-i3wm - Jupyter Notebook - Google Chrome',
  'floating': False},
 {'id': 94648427394704,
  'x': 2,
  'y': 31,
  'width': 1276,
  'height': 767,
  'cx': 640,
  'cy': 414,
  'name': '/home/user/develop/working/computervision/pysight/notebooks/combining-headpose-eye-location-for-gaze-estimation.pdf',
  'floating': False},
 {'id': 94648427255968,
  'x': 109,
  'y': 133,
  'width': 1123,
  'height': 538,
  'cx': 670,
  'cy': 402,
  'name': 'tmux',
  'floating': True}]

In [8]:
# Intersection function
def get_intersecting_windows(x, y, windows):
    intersections = []
    for w in windows:
        if (w['x'] <= x <= (w['x'] + w['width']) and
            w['y'] <= y <= (w['y'] + w['height'])):
                intersections.append(w)
    return intersections

In [14]:
# Distance function
from math import sqrt

def distance_to_query(x, y, window):
    dx   = abs(window['cx'] - x) ** 2
    dy   = abs(window['cy'] - y) ** 2
    dist = int(sqrt(dx + dy))
    return dist

def get_closest_window(x, y, windows):
    if len(windows) == 0:
        raise Exception('No windows given')
        
    elif len(windows) == 1:
        return windows[0]
    
    else:  
        closest_so_far, dist = None, float('inf')

        for w in windows:
            w_dist = distance_to_query(x, y, w)
            if w_dist <= dist:
                closest_so_far = w
                dist = w_dist

        return closest_so_far

In [10]:
# rtree?

In [11]:
# scipy Voronoi?

### Algorithm

Definitions:

- X<sub>query</sub>, Y<sub>query</sub>
- Width<sub>screen</sub>, Height<sub>screen</sub>


Steps:

1. Get dimensions of currently focused I3 output
2. Check x, y is within dimensions. If not raise error/return
3. Get windows of current output workspace
4. Filter non-visible windows (and currently focused window???)
5. Extract information from windows
6. Return windows that contain `X, Y`
7. Return id of window who's center is closest to Query.


Extra:
1. Build spatial index to ease the intersection
2. 

In [21]:
def main(x, y):
    i3                   = i3ipc.Connection()
    width, height        = get_ws_dimensions(x, y, i3)
    windows              = get_ws_windows(i3)
    window_info          = get_win_boxes(windows)
    window_intersections = get_intersecting_windows(x, y, window_info)
    closest_window       = get_closest_window(x, y, window_intersections)
    return closest_window

In [24]:
main(300, 300)

{'id': 94648427380528,
 'x': 0,
 'y': 29,
 'width': 1280,
 'height': 771,
 'cx': 640,
 'cy': 414,
 'name': '03-i3wm - Jupyter Notebook - Google Chrome',
 'floating': False}